In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Generic imports
import numpy as np
from numpy.testing import assert_allclose
import matplotlib.pyplot as plt
from ase import Atoms
from scipy.interpolate import make_interp_spline

# Imports specific to pyLODE
from pylode.lib.projection_coeffs import DensityProjectionCalculator
from pylode.lib.projection_coeffs_realspace import DensityProjectionCalculatorRealspace

In [ ]:
xx = np.linspace(0, 10, 1000)
yy = np.cos(3*xx)**2 + np.exp(0.2 * xx)
plt.plot(xx, yy)

In [ ]:
spl = make_interp_spline(xx, yy, 3)
assert_allclose(spl(xx), yy, rtol=1e-6)
plt.plot(xx, spl(xx), 'r')

In [ ]:
plt.plot(xx, spl(xx, 1))
dy = -6 * np.sin(3*xx) * np.cos(3*xx) + 0.2 * np.exp(0.2*xx)
plt.plot(xx, dy)
assert_allclose(dy, spl(xx,1), rtol=2e-3)

# Compare real space implementation against reciprocal space

In [ ]:
frames = []
cell = np.eye(3) * 16
Ndimers = 15
distances = np.linspace(1., 2.5, Ndimers)
for d in distances:
    positions = [[1,1,1],[1,1,1+d]]
    frame = Atoms('O2', positions=positions, cell=cell, pbc=True)
    frames.append(frame)

In [ ]:
nmax = 8
lmax = 3
rcut = 6.
smearing = 0.7

In [ ]:
hypers = {
    'smearing':smearing,
    'max_angular':lmax,
    'max_radial':nmax,
    'cutoff_radius':rcut,
    'potential_exponent':0,
    'radial_basis': 'gto',
    'compute_gradients':False,
    'subtract_center_contribution':False,
    }

calculator_pylode = DensityProjectionCalculator(**hypers)
calculator_pylode.transform(frames)
features_pylode = calculator_pylode.features

In [ ]:
calculator_realspace = DensityProjectionCalculatorRealspace(**hypers)
calculator_realspace.transform(frames)
features_realspace = calculator_realspace.features

In [ ]:
np.round(features_pylode[0,0,:,1:], 8)

In [ ]:
np.round(features_realspace[0,0,:,1:], 8)

In [ ]:
hypers_realspace = {
    'smearing':smearing,
    'max_angular':lmax,
    'max_radial':1,
    'cutoff_radius':rcut,
    'potential_exponent':1,
    'radial_basis': 'gto',
    'compute_gradients':False,
    'subtract_center_contribution':False,
    }

In [ ]:
calculator_realspace = DensityProjectionCalculatorRealspace(**hypers_realspace)
calculator_realspace.transform(frames)
features_realspace = calculator_realspace.features